In [2]:
import numpy as np
import pandas as pd
import os
import random
import gym
import pylab
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, Lambda, Add, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import backend as K
import cv2
from NetworkEnv import NetworkEnv as Ne

In [43]:
x = "name"
test = Ne(x)
print(type(test.state_observation))

<class 'numpy.ndarray'>


In [7]:
data = np.array([[3,4,1,2,3],[2,5,1,5,1],[3,5,2,5,3],[4,3,5,1,2],[4,3,2,1,4],[2,1,4,5,3],[5,5,2,2,1],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]])
df = pd.DataFrame(data, columns=["N1Q","N2Q","N3Q","N4Q","N5Q"])

In [44]:
test.reset(df)
print(test.state_observation)

[[3 4 1 2 3]
 [2 5 1 5 1]
 [3 5 2 5 3]
 [4 3 5 1 2]
 [4 3 2 1 4]
 [2 1 4 5 3]
 [5 5 2 2 1]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]


In [46]:
one_hot_state = preprocess(test.state_observation)
print(one_hot_state)
print(one_hot_state.shape)

[[[0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 1. 0.]
  [1. 0. 0. 0. 1.]
  [0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 1.]
  [1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 1. 0. 1. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [1. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0.]
  [0. 1. 0. 1. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 1.]
  [0. 1. 0. 0. 0.]
  [1. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0.]
  [0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0.]
  [1. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0.]
  [1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 1. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1.]
  [0. 0. 1. 1. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [1. 1. 0. 0. 0.]]

 [[1. 1. 1. 1. 1.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[1. 1. 1. 1. 1.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 

In [47]:
testactor, testcritic  = OurModel([10,6,5], [3,4,4,4,4], 0.00025)


KerasTensor(type_spec=TensorSpec(shape=(None, 10, 6, 5), dtype=tf.float32, name='input_17'), name='input_17', description="created by layer 'input_17'")


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [52]:
state =  tf.convert_to_tensor([one_hot_state])
prob = testactor.predict(state) 
action = np.zeros(5)
count = 0
for i in prob:
  print(i)
  for j in i:
    print(j)
    action[count] = np.random.choice(len(j),p=j)
    count +=1
print(action)
values = testcritic.predict(state)
print(values)

[[0.37103254 0.3275475  0.30142003]]
[0.37103254 0.3275475  0.30142003]
[[0.4119788  0.16134168 0.2126781  0.21400145]]
[0.4119788  0.16134168 0.2126781  0.21400145]
[[0.13933189 0.25303352 0.2200966  0.38753796]]
[0.13933189 0.25303352 0.2200966  0.38753796]
[[0.2045208  0.19550997 0.19996275 0.40000647]]
[0.2045208  0.19550997 0.19996275 0.40000647]
[[0.37628692 0.20249857 0.20806566 0.2131488 ]]
[0.37628692 0.20249857 0.20806566 0.2131488 ]
[2. 0. 2. 1. 3.]
[array([[-0.47134167]], dtype=float32), array([[0.17622513]], dtype=float32), array([[-0.3048492]], dtype=float32), array([[0.18241327]], dtype=float32), array([[0.05048151]], dtype=float32)]


In [3]:
def preprocess(new_state):
  q1 = new_state[:,0]
  q2 = new_state[:,1]
  q3 = new_state[:,2]
  q4 = new_state[:,3]
  q5 = new_state[:,4]
  q1 =np.array(q1)
  ohq1 = np.zeros((q1.shape[0], 6))
  ohq1[np.arange(q1.size),q1]=1

  q2 =np.array(q2)
  ohq2 = np.zeros((q2.shape[0], 6))
  ohq2[np.arange(q2.size),q2]=1
  

  q3 =np.array(q3)
  ohq3 = np.zeros((q3.shape[0], 6))
  ohq3[np.arange(q3.size),q3]=1

  q4 =np.array(q4)
  ohq4 = np.zeros((q4.shape[0], 6))
  ohq4[np.arange(q4.size),q4]=1

  q5 =np.array(q5)
  ohq5 = np.zeros((q5.shape[0], 6))
  ohq5[np.arange(q5.size),q5]=1

  ns = np.dstack((ohq1,ohq2,ohq3,ohq4,ohq5))
  return ns


def OurModel(state_shape, action_space, lr):
  X_input = Input(state_shape)
  print(X_input)
  X = Conv2D(32, 3, activation = "relu", input_shape=(state_shape))(X_input)
  X = MaxPooling2D((2,2))(X)
  X = Flatten()(X)

  action_1 = Dense(action_space[0], activation="softmax", kernel_initializer='he_uniform')(X)
  action_2 = Dense(action_space[1], activation="softmax", kernel_initializer='he_uniform')(X)
  action_3 = Dense(action_space[2], activation="softmax", kernel_initializer='he_uniform')(X)
  action_4 = Dense(action_space[3], activation="softmax", kernel_initializer='he_uniform')(X)
  action_5 = Dense(action_space[4], activation="softmax", kernel_initializer='he_uniform')(X)

  value_1 = Dense(1, kernel_initializer='he_uniform')(X)
  value_2 = Dense(1, kernel_initializer='he_uniform')(X)
  value_3 = Dense(1, kernel_initializer='he_uniform')(X)
  value_4 = Dense(1, kernel_initializer='he_uniform')(X)
  value_5 = Dense(1, kernel_initializer='he_uniform')(X)

  Actor = Model(inputs = X_input, outputs = [action_1, action_2, action_3, action_4, action_5])  
  Actor.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=lr))


  Critic = Model(inputs = X_input, outputs = [value_1, value_2, value_3, value_4, value_5])
  Critic.compile(loss='categorical_crossentropy', optimizer=Adam(lr=lr))



  return Actor, Critic



In [35]:
class A2CAgent:
    # Policy Gradient Main Optimization Algorithm
    def __init__(self, env_name):
        self.env_name = env_name       
        self.env = Ne(env_name)
        self.action_size = [self.env.N1_max_actions+1, self.env.N2_max_actions+1, self.env.N3_max_actions+1, self.env.N4_max_actions+1, self.env.N5_max_actions+1]
        self.EPISODES, self.max_average = 2000, -1000000.0
        self.lr = 1

        self.max_Q = 10
        self.Nodes = 5
        self.one_hot_dim = 6

        self.Save_Path = 'Models'

        
        self.states, self.actions, self.rewards = [], [], []
        self.scores, self.episodes, self.average = [], [], []

        self.state_size = [self.max_Q, self.one_hot_dim, self.Nodes]
        
        if not os.path.exists(self.Save_Path): os.makedirs(self.Save_Path)
        self.path = '{}_PG_{}'.format(self.env_name, self.lr)
        self.Model_name = os.path.join(self.Save_Path, self.path)


        self.Actor, self.Critic = OurModel(self.state_size, self.action_size, self.lr)


    def load(self, Actor_name):
        self.Actor = load_model(Actor_name, compile=False)

    def save(self):
        print(self.Model_name)
        self.Actor.save(self.Model_name + '.h5')
        


    def remember(self, state, action, reward):
        # store episode actions to memory
        self.states.append(state)
        self.actions.append(action)
        self.rewards.append(reward)
 

    def act(self, state):
        # Use the network to predict the next action to take, using the model
        one_hot_state = preprocess(state)
        one_hot_state = tf.convert_to_tensor([one_hot_state]) 
        prediction = self.Actor.predict(one_hot_state)
        action = np.zeros(5)
        count = 0
        for i in prediction:
          for j in i:
            action[count] = np.random.choice(len(j),p=j)
            count +=1
        return action


    def replay(self):
        # reshape memory to appropriate shape for training
        states = np.vstack(self.states)
        actions = np.vstack(self.actions)

        # Compute discounted rewards
        discounted_r = self.discount_rewards(self.rewards)

        # Get Critic network predictions
        values = self.Critic.predict(states)[:, 0]
        # Compute advantages
        advantages = discounted_r - values
        # training Actor and Critic networks
        self.Actor.fit(states, actions, sample_weight=advantages, epochs=1, verbose=0)
        self.Critic.fit(states, discounted_r, epochs=1, verbose=0)
        # reset training memory
        self.states, self.actions, self.rewards = [], [], []

    def discount_rewards(self, reward):
    # Compute the gamma-discounted rewards over an episode
        gamma = 0.99    # discount rate
        running_add = 0
        discounted_r = np.zeros_like(reward)
        for i in reversed(range(0,len(reward))):
            running_add = running_add * gamma + reward[i]
            discounted_r[i] = running_add

        discounted_r = discounted_r - np.mean(discounted_r) # normalizing the result
        discounted_r = discounted_r/np.std(discounted_r) # divide by standard deviation
        return discounted_r
   
    def PlotModel(self, score, episode):
        self.scores.append(score)
        self.episodes.append(episode)
        self.average.append(sum(self.scores[-50:]) / len(self.scores[-50:]))
        if str(episode)[-2:] == "00":# much faster than episode % 100
            pylab.plot(self.episodes, self.scores, 'b')
            pylab.plot(self.episodes, self.average, 'r')
            pylab.ylabel('Score', fontsize=18)
            pylab.xlabel('Steps', fontsize=18)
            try:
                pylab.savefig(self.path+".png")
            except OSError:
                pass

        return self.average[-1]

    def reset(self, df):
        self.env.reset(df)
        state = self.env.state_observation
        return state

    def step(self,action):
        next_state, reward, done, info = self.env.step(action)
        return next_state, reward, done, info

    def run(self, df):

        for e in range(self.EPISODES):
          state = self.reset(df)
          e = 0
          
          done, score, SAVING = False, 0, ''
          while not done:
              #self.env.render()
              # Actor picks an action
              action = self.act(state)
              # Retrieve new state, reward, and whether the state is terminal
              next_state, reward, done, e = self.step(action)
              # Memorize (state, action, reward) for training
              self.remember(state, action, reward)
              # Update current state
              state = next_state
              score += reward
              
              if done:
                  average = self.PlotModel(score, self.env.episode_length)
                  # saving best models
                  if average >= self.max_average:
                      self.max_average = average
                      self.save()
                      SAVING = "SAVING"
                  else:
                      SAVING = ""
                  print("episode: {}/{}, score: {}, average: {:.2f} {}".format(e, self.EPISODES, score, average, SAVING))

                  self.replay()

    def test(self, Model_name,df):
        self.load(Model_name)
        for e in range(100):
          state = self.reset(df)
          done = False
          score = 0
          while not done:
            one_hot_state = preprocess(state)
            one_hot_state = tf.convert_to_tensor([one_hot_state]) 
            prediction = self.Actor.predict(one_hot_state)
            action = np.zeros(5)
            count = 0
            for i in prediction:
              for j in i:
                action[count] = np.argmax(j)
                count +=1
            state, reward, done, _ = self.step(action)
            score += reward
            print(state)
            print(action)
            print(reward)
            if done:
              print("episode: {}/{}, score: {}".format(e, self.EPISODES, score))
              print(average)
              break
        
      # close environemnt when finish training


In [39]:
    env_name = 'Nettest1'
    agent = A2CAgent(env_name)
    agent.run(df)

KerasTensor(type_spec=TensorSpec(shape=(None, 10, 6, 5), dtype=tf.float32, name='input_15'), name='input_15', description="created by layer 'input_15'")


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Models/Nettest1_PG_1
episode: 43/2000, score: -670, average: -670.00 SAVING


ValueError: ignored

In [37]:
agent.test("/content/Models/Nettest1_PG_1.h5",df)

[[2 5 2 3 3]
 [3 5 4 1 1]
 [4 3 1 5 3]
 [4 3 2 5 2]
 [2 1 5 1 4]
 [5 5 2 1 3]
 [0 0 4 5 1]
 [0 0 2 2 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
[2. 2. 2. 2. 3.]
-35
[[2 5 5 3 3]
 [3 3 4 1 1]
 [4 3 1 5 3]
 [4 1 2 5 2]
 [2 5 5 1 4]
 [5 0 2 1 3]
 [0 0 4 5 1]
 [0 0 2 2 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
[0. 2. 1. 1. 3.]
-23
[[2 5 5 3 3]
 [3 3 4 1 1]
 [4 3 1 5 3]
 [4 1 2 5 2]
 [2 5 5 1 4]
 [5 0 2 1 3]
 [0 0 4 5 1]
 [0 0 2 2 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
[0. 2. 1. 1. 3.]
-38
[[2 5 5 3 3]
 [3 3 4 1 1]
 [4 3 1 5 3]
 [4 1 2 5 2]
 [2 5 5 1 4]
 [5 0 2 1 3]
 [0 0 4 5 1]
 [0 0 2 2 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
[0. 2. 1. 1. 3.]
-38
[[2 5 5 3 3]
 [3 3 4 1 1]
 [4 3 1 5 3]
 [4 1 2 5 2]
 [2 5 5 1 4]
 [5 0 2 1 3]
 [0 0 4 5 1]
 [0 0 2 2 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
[0. 2. 1. 1. 3.]
-38
[[2 5 5 3 3]
 [3 3 4 1 1]
 [4 3 1 5 3]
 [4 1 2 5 2]
 [2 5 5 1 4]
 [5 0 2 1 3]
 [0 0 4 5 1]
 [0 0 2 2 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
[0. 2. 1. 1. 3.]
-38
[[2 5 5 3 3]
 [3 3 4 1 1]
 [4 3 1 5 3]
 [4 1 2 5 2]
 [2 5 5 1 4]
 [5 0 2 1 3]
 [0 0 4 5 

NameError: ignored

In [6]:
for i in range(0,30):
  agent.run(df)

episode: 110/2000, score: -970, average: -1048.00 


UFuncTypeError: ignored